In [7]:
# set up the data set
from datetime import time
import pandas as pd

data = pd.read_excel(r"newInput.xlsx")
df = pd.DataFrame(data)

In [15]:
"""defining functions to converts to seconds and then  back to time object """
def convertToMin(time_obj):
    return (time_obj.hour * 60 + time_obj.minute)

def convertToTime(minutes):
    hour, minute = divmod(minutes, 60)
    return time(hour, minute)

In [16]:
# can create a additional function to call this in future development 
strToTime = lambda strTime: time(*[int(x) for x in strTime.split(':')])
# DEFINING GLOBAL VARIABLES 
sorted_set = df.sort_values('Departure Time', ascending=True).reset_index(drop=True)
start_set = sorted_set[sorted_set['Departure Place'] == 'PSL'].reset_index(drop=True)

# GLOBAL VARIABLES
# all the time constraints are mentioned in minutes
minDuty = 8 * 60 
maxDuty = 10 * 60 
# minSpreadOver = 10 * 60 
maxSpreadOver = 12 * 60 
trip = [{"selected_leg":start_set.iloc[0].to_frame().T, 
         "temp_set": start_set.drop(0).reset_index(drop=True),
         }]


In [35]:

def getCurrentTrip():
    global trip
    route = [x['selected_leg'] for x in trip]
    display(pd.concat(route).reset_index(drop=True))

def getSpreadOver():
    global trip
    start = strToTime(trip[0]['selected_leg']['Departure Time'][0])
    end = strToTime(trip[-1]['selected_leg']['Arrival Time act'][0])
    return (convertToMin(end) - convertToMin(start))


In [26]:
def pushToTrip():
    global trip
    while trip[-1]['selected_leg']['Arrival Place'][0] != 'PSL':
        last_leg = trip[-1]['selected_leg']
        temp_set = df[(df['Departure Place'] == last_leg['Arrival Place'][0]) & (df['Departure Time'] > last_leg['Arrival Time act'][0])]
        temp_set = temp_set.sort_values('Departure Time', ascending=True ).reset_index(drop=True)
        new_leg = temp_set.iloc[0].to_frame().T
        trip.append({"selected_leg": new_leg,
                 "temp_set": temp_set.drop(0).reset_index(drop=True)})

pushToTrip()
getCurrentTrip()

,Sl No.,Departure Time,Departure Place,Route of Operation,Arrival Place,Arrival Time act,KM,Running Time
0,1,03:40:00,PSL,NH,KLKV,03:45:00,3.5,00:05:00
1,124,03:50:00,KLKV,NH,TVM,04:55:00,33.7,01:05:00
2,313,05:05:00,TVM,NH-UDA,KNVLA,06:05:00,32.0,01:00:00
3,641,06:15:00,KNVLA,UDA,MC,08:00:00,39.0,01:45:00
4,336,08:10:00,MC,NH,KLKV,10:00:00,40.0,01:50:00
5,695,10:05:00,KLKV,PVR-VZM-BYPASS-TVM-NH-KLKV,PSL,13:35:00,81.5,03:30:00


In [ ]:
for i in range(50):
    # trip.pop()
    trip[-1]['selected_leg'] = trip[-1]['temp_set'].iloc[0].to_frame().T
    trip[-1]['temp_set'] = (trip[-1]['temp_set'].drop(0)).reset_index(drop=True)
    pushToTrip()
    print("Trip ", i)
    getCurrentTrip()

In [ ]:

""" 
Sl No.
Departure Time
Departure Place
Route of Operation
Arrival Place
Arrival Time act
KM
Running Time 
"""